In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize

import pandas as pd
import numpy as np
from numpy import linalg as LA
import math
import matplotlib.pyplot as plt

Raw_Data = pd.read_csv('vehicles-processed.csv')


Raw_Data_temp = np.array(Raw_Data.drop(columns=['make','model','VClass','gears']))
display(Raw_Data)

# np.random.shuffle(Raw_Data_temp)
X = Raw_Data_temp[:,1:]
Y = Raw_Data_temp[:,0]

print(X)

# for i in range(len(X[1])):
#   X[:,i] = X[:,i] / np.linalg.norm(X[:,i])

X_Train = X[:int(len(X)*0.66)]
X_Test = X[int(len(X)*0.66):]

Y_Train = Y[:int(len(X)*0.66)]
Y_Test = Y[int(len(X)*0.66):]

,make,model,VClass,comb08,cylinders,displ,drive,year,sCharger,tCharger,gearbox,gears
0,Alfa Romeo,Spider Veloce 2000,Two Seaters,21,4,2.0,2,1985,0,0,0,5
1,Ferrari,Testarossa,Two Seaters,11,12,4.9,2,1985,0,0,0,5
2,Dodge,Charger,Subcompact Cars,27,4,2.2,2,1985,0,0,0,5
3,Dodge,B150/B250 Wagon 2WD,Vans,11,8,5.2,2,1985,0,0,1,3
4,Subaru,Loyale,Compact Cars,22,4,1.8,2,1993,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
28739,Saturn,SL,Compact Cars,25,4,1.9,2,1993,0,0,0,5
28740,Subaru,Legacy,Compact Cars,22,4,2.2,2,1993,0,0,1,4
28741,Subaru,Legacy,Compact Cars,23,4,2.2,2,1993,0,0,0,5
28742,Subaru,Legacy AWD,Compact Cars,21,4,2.2,4,1993,0,0,1,4


[[ 4.   2.   2.  ...  0.   0.   0. ]
 [12.   4.9  2.  ...  0.   0.   0. ]
 [ 4.   2.2  2.  ...  0.   0.   0. ]
 ...
 [ 4.   2.2  2.  ...  0.   0.   0. ]
 [ 4.   2.2  4.  ...  0.   0.   1. ]
 [ 4.   2.2  4.  ...  0.   0.   0. ]]


In [25]:
reg = LinearRegression().fit(X_Train, Y_Train)
Y_predict = reg.predict(X_Train)

Titles = Raw_Data.drop(columns=['comb08','make','model','VClass','gears']).columns.tolist()
Coefficient = pd.DataFrame(reg.coef_).T
Coefficient.columns=Titles
display(Coefficient)

from sklearn.metrics import mean_squared_error
print(mean_squared_error(Y_Train, Y_predict))

print("\n\n")
for i in range(10):
  X_Value = pd.DataFrame(X_Train[i],Titles).T
  display(X_Value)
  print("True_Value: ",round(Y_Train[i],2),"Predicted_Value: ",round(Y_predict[i],2))
  print()

,cylinders,displ,drive,year,sCharger,tCharger,gearbox
0,-0.244759,-2.753304,-0.950004,0.156676,-0.156767,-1.765336,-0.595258


8.396198094277217





,cylinders,displ,drive,year,sCharger,tCharger,gearbox
0,4.0,2.0,2.0,1985.0,0.0,0.0,0.0


True_Value:  21.0 Predicted_Value:  22.36



,cylinders,displ,drive,year,sCharger,tCharger,gearbox
0,12.0,4.9,2.0,1985.0,0.0,0.0,0.0


...

True_Value:  26.0 Predicted_Value:  24.72



,cylinders,displ,drive,year,sCharger,tCharger,gearbox
0,4.0,1.8,2.0,1993.0,0.0,0.0,1.0


True_Value:  25.0 Predicted_Value:  23.57



,cylinders,displ,drive,year,sCharger,tCharger,gearbox
0,4.0,1.8,2.0,1993.0,0.0,0.0,0.0


True_Value:  26.0 Predicted_Value:  24.17



In [23]:
epsilon = 0.5
delta = 0.5

X_Size = 500

# np.random.shuffle(Raw_Data_temp)
X = Raw_Data_temp[:,1:]
Y = Raw_Data_temp[:,0]

for i in range(len(X[1])):
  X[:,i] = X[:,i] / np.linalg.norm(X[:,i])

X_Train = X[:X_Size]
X_Train2 = X[X_Size:X_Size*2]

Y_Train = Y[:X_Size]
Y_Train2 = Y[X_Size:X_Size*2]

reg = LinearRegression().fit(X_Train, Y_Train)


C = 1
Q = (epsilon**2)/(X_Size*np.sqrt(np.log(1/(epsilon*delta)))*np.log(X_Size))
alpha1 = C*Q*np.log(Q)
alpha2 = epsilon/4

# for t in range(len(X_Train2)):
for t in range(5):
  xt = X_Train2[t]
  U, S, Vh = LA.svd(np.matmul(X_Train2.transpose(),X_Train2), full_matrices=False)
  # S = np.diag(S)

  # U = np.matrix(U)
  # S = np.matrix(S)
  # V = np.matrix(Vh)

  # q = X_Train2@U.getH()
  # q = q@LA.inv(S.getH())
  # q = q@U.getH().transpose()
  # q = q@xt

  q = xt@LA.pinv(X_Train2.transpose()@X_Train2)@X_Train2.transpose()
  # print(np.shape(q))
  # print(xt-q@X_Train2)

  u = U.transpose()@xt

  print(LA.norm(q),LA.norm(u))
  print()
  if(LA.norm(q)<= alpha1 and LA.norm(u)<= alpha2):
    Y_predict = reg.predict(xt)

  else:

    Y_predict = "Refect"
    yt = Y_Train2[t]
    X_Train = np.delete(X_Train, 0,  axis=0)
    Y_Train = np.delete(Y_Train, 0,  axis=0)

    X_Train = np.append(X_Train, xt, axis=0)
    Y_Train = np.append(Y_Train, yt, axis=0)

    reg = LinearRegression().fit(X_Train, Y_Train)






## Need to figure the dimansion of q and v vectors



0.12433119143776662 0.016183021063869964

0.15933527190018412 0.016913900917955412

0.09436541407737307 0.009580612177721883

0.09267343599818695 0.013995932425535583

0.11231399658298416 0.012030704285275341



In [3]:
X_Train2.transpose()@X_Train2

NameError: ignored

In [ ]:
# print(np.shape(U))
# print(np.shape(S))
# print(np.shape(Vh))
# print(np.shape(np.matmul(X_Train2,X_Train2.transpose())))
# print(np.shape(np.matmul(np.matmul(U,np.diag(S)),Vh)))

# print(np.matmul(X_Train2,X_Train2.transpose()))
# print(np.matmul(np.matmul(U,np.diag(S)),Vh))

a = np.array([2,3]).transpose()
b = np.array([[1,0],[0,1]])
print(np.shape(a))
print(np.shape(b))

ans = np.matmul(b,a)
np.matrix(ans).getH()

print(b@a)
print(ans)

(2,)
(2, 2)
[2 3]
[2 3]
